paraphrasing - backtranslation nlpaug\
noise injection - word, grammar, irrelevant\
random augmentation\
\
filtering later on


!pip install -q nlpaug
!pip install sacremoses

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [8]:
question = "The weather is lovely, isn't it?"

### Back-translation

In [6]:
import nlpaug.augmenter.word as naw

In [7]:
def backtranslate (query: str) -> str:
    aug = naw.back_translation.BackTranslationAug(
        from_model_name="Helsinki-NLP/opus-mt-en-de",
        to_model_name="Helsinki-NLP/opus-mt-de-en",
    )

    augmented_query = aug.augment(query)
    return augmented_query

In [126]:
backtranslate(question)

["The weather is nice, isn't it?"]

----

### Noise injection

word\
grammar\
irrelevant

#### Word-level: typos and letter-swaps

In [53]:
import nlpaug.augmenter.char as nac
from nlpaug.flow import Sequential

In [45]:
aug_typo = nac.KeyboardAug(
    aug_word_p=0.2,
    aug_char_p=0.1,
    aug_char_min=1,
    aug_char_max=2,
    include_special_char=False,
    include_numeric=False,
    include_upper_case=False,
)

In [49]:
aug_swapletter = nac.RandomCharAug(
    action='swap',
    aug_word_p=0.2,
    aug_char_p=0.1,
    aug_char_min=1,
    aug_char_max=2,
)

In [54]:
pipeline_word = Sequential([aug_typo, aug_swapletter])

In [56]:
print('Typo:', aug_typo.augment(question))
print('Letter-swap:', aug_swapletter.augment(question))
print('Combined:', pipeline_word.augment(question))

Typo: ["The weatjer is loveiy, isn ' t it?"]
Letter-swap: ["The weatehr is lvoely, isn ' t it?"]
Combined: ["The waethdr is olveiy, isn ' t it?"]


#### Irrelevant information

pip install wikipedia

pip install wikipedia-api

In [105]:
import wikipedia, random
import wikipediaapi

wiki = wikipediaapi.Wikipedia("ShrinkMCP kanecorts@gmail.com", "en")

In [122]:
def get_random_sentence() -> str:
    title = wikipedia.random(pages=1)
    page = wiki.page(title)
    
    if not page.exists():
        return ""
    
    # split summary into sentences
    sentences = page.summary.split(". ")
    random_sentence = random.choice(sentences) + '.'
    return random_sentence.strip()

In [121]:
random_sentence = get_random_sentence()
augmented_question = random_sentence + ' ' + question
print(augmented_question)

He participated in the British attack on Cádiz in 1587. The weather is lovely, isn't it?


---

### Random augmentation

pip install nltk

In [9]:
import nltk

# nltk.download('wordnet')
# nltk.download('omw-1.4') 
# nltk.download('averaged_perceptron_tagger')
# nltk.download('averaged_perceptron_tagger_eng')

In [56]:
import re
def regex_tokenizer(text):
    return re.findall(r"\w+(?:'\w+)?|[^\w\s]", text)

In [128]:
def synonym_replacement(query: str) -> str:
    syn_aug = naw.synonym.SynonymAug(
        aug_src='wordnet', 
        aug_p=0,
        aug_max=5,
        tokenizer=regex_tokenizer,
        stopwords=['it', 'as']
    )
    synonym_text = syn_aug.augment(query)
    
    return synonym_text

In [125]:
question = "The weather is lovely, isn't it?"

In [129]:
synonym_replacement(question)

["The atmospheric condition be lovely, isn't it?"]

In [130]:
def random_substitution(query: str) -> str:
    sub_aug = naw.random.RandomWordAug(
        action='substitute', 
        aug_max=5,
        tokenizer=regex_tokenizer
    )
    subsituted_text = sub_aug.augment(query)

    return subsituted_text

In [131]:
random_substitution(question)

["_ _ is lovely, isn't _?"]